In [0]:
import pandas as pd
import pymssql
import pymysql
import pyspark.sql.functions as F

In [0]:
jdbcHostname = "54.245.193.229" #"54.245.193.229" 
jdbcDatabase = "DEV" #"KAGGLE" 
jdbcPort = "1433" #1433 
jdbcUsername = "sa"
jdbcPassword = "WindowsSQL-123"

host = "db4free.net"
db = "exocube"
uname = "sample_jenish"
pas = "Jenu7492@patel"

In [0]:
Source_Table_Name = "EMP_INFO"

In [0]:
def get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase):
    try:
        sql_connection = pymssql.connect(host=jdbcHostname,user=jdbcUsername,password=jdbcPassword,database=jdbcDatabase, as_dict=True, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymssql.OperationalError as e:
        print("connection failed")
    except:
        print("Unknown error occurred")
    else:
        return sql_connection

In [0]:
def get_myconnection(host, uname, pas, db):
    try:
        sql_myconnection = pymysql.connect(host=host,user=uname,passwd=pas,db=db, cursorclass=pymysql.cursors.DictCursor, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymysql.OperationalError as e:
        if e.args[0] == 2003:
            print('server not found')
        elif e.args[0] == 1044:
            print('database not found')
        elif e.args[0] == 1045:
            print('username or password incorrect')
    except:
        print("Unknown error occurred")   
    else:
        return sql_myconnection

In [0]:
def display_mssql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        cursor.execute(sql_disqry)
        count = cursor.rowcount
        if count:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            display(result)
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
def sql_auto_gen_qry(qry, sql_conn):
    try:
        adres = 'select '
        cursor = sql_conn.cursor()
        cursor.execute(qry)
        count = cursor.rowcount
        if count:
            sql_data = cursor.fetchall()
            for val in sql_data:
                dt,col = val.values()
                if dt == 'int':
                    qry = "count("+col+") as "+col+"CNT,min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,avg("+col+")as "+col+"AVG,"
                elif dt == 'varchar':
                    qry = "count(distinct(trim(lower("+col+")))) as "+col+"CNT,min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(len(trim(lower("+col+"))))as "+col+"SUM,avg(len(trim(lower("+col+"))))as "+col+"AVG,"
                else:
                    qry = "count(DATEDIFF(s, '1970-01-01 00:00:00', "+col+")) as "+col+"CNT,min(DATEDIFF(s, '1970-01-01 00:00:00', "+col+"))as "+col+"MIN,max(DATEDIFF(s, '1970-01-01 00:00:00', "+col+"))as "+col+"MAX,sum(DATEDIFF(s, '1970-01-01 00:00:00', "+col+") * convert(bigint, 1000))as "+col+"SUM,avg(DATEDIFF(s, '1970-01-01 00:00:00', "+col+") * convert(bigint, 1000))as "+col+"AVG,"
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)

In [0]:
qry ="(SELECT DATA_TYPE, COLUMN_NAME FROM information_schema.columns where table_name = '{}')".format(Source_Table_Name)
sql_disqry = sql_auto_gen_qry(qry, sql_conn)

In [0]:
display_mssql_data(sql_disqry, sql_conn)

Emp_IdCNT,Emp_IdMIN,Emp_IdMAX,Emp_IdSUM,Emp_IdAVG,Emp_nameCNT,Emp_nameMIN,Emp_nameMAX,Emp_nameSUM,Emp_nameAVG,DesignationCNT,DesignationMIN,DesignationMAX,DesignationSUM,DesignationAVG,DepartmentCNT,DepartmentMIN,DepartmentMAX,DepartmentSUM,DepartmentAVG,CityCNT,CityMIN,CityMAX,CitySUM,CityAVG,SalaryCNT,SalaryMIN,SalaryMAX,SalarySUM,SalaryAVG,Joining_DateCNT,Joining_DateMIN,Joining_DateMAX,Joining_DateSUM,Joining_DateAVG
15,4,21,194,12,15,amit desai,suresh ahir,175,11,6,admin,manager,85,5,2,hr,it,30,2,4,bardoli,surat,87,5,15,18690,56000,497284,33152,15,978307200,1427846400,18399657600000,1226643840000


In [0]:
def display_mysql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        sql_exec = cursor.execute(sql_disqry)
        if sql_exec:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            display(result)
        else:
            print(sql_exec)
            print("No Record")
    except pymysql.ProgrammingError as e:
        if e.args[0] == 1146:
            print('table doesnt exist')
        elif e.args[0] == 1064:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
def mysql_auto_gen_qry(qry, sql_conn):
    try:
        adres = 'select '
        cursor = sql_conn.cursor()
        cursor.execute(qry)
        count = cursor.rowcount
        if count:
            sql_data = cursor.fetchall()
            for val in sql_data:
                dt,col = val.values()
                if dt == 'int':
                    qry = "count("+col+") as "+col+"CNT,min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,FLOOR(avg("+col+"))as "+col+"AVG,"
                elif dt == 'varchar':
                    qry = "count(distinct(trim(lower("+col+")))) as "+col+"CNT,min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(length(trim(lower("+col+"))))as "+col+"SUM,FLOOR(avg(length(trim(lower("+col+")))))as "+col+"AVG,"
                else:
                    qry = "count(UNIX_TIMESTAMP("+col+")) as "+col+"CNT,min(UNIX_TIMESTAMP("+col+"))as "+col+"MIN,max(UNIX_TIMESTAMP("+col+"))as "+col+"MAX,sum(UNIX_TIMESTAMP("+col+"))as "+col+"SUM,FLOOR(avg(UNIX_TIMESTAMP("+col+")))as "+col+"AVG,"
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_myconnection(host, uname, pas, db)

In [0]:
qry ="(SELECT DATA_TYPE, COLUMN_NAME FROM information_schema.columns where table_name = '{}')".format(Source_Table_Name)
sql_disqry = mysql_auto_gen_qry(qry, sql_conn)

In [0]:
display_mysql_data(sql_disqry, sql_conn)

Emp_IdCNT,Emp_IdMIN,Emp_IdMAX,Emp_IdSUM,Emp_IdAVG,Emp_nameCNT,Emp_nameMIN,Emp_nameMAX,Emp_nameSUM,Emp_nameAVG,DesignationCNT,DesignationMIN,DesignationMAX,DesignationSUM,DesignationAVG,DepartmentCNT,DepartmentMIN,DepartmentMAX,DepartmentSUM,DepartmentAVG,CityCNT,CityMIN,CityMAX,CitySUM,CityAVG,SalaryCNT,SalaryMIN,SalaryMAX,SalarySUM,SalaryAVG,Joining_DateCNT,Joining_DateMIN,Joining_DateMAX,Joining_DateSUM,Joining_DateAVG
15,4,21,194,12,15,amit desai,suresh ahir,175,11,6,admin,manager,85,5,2,hr,it,30,2,4,bardoli,surat,87,5,15,18690,56000,497284,33152,15,978307200,1427846400,18399657600,1226643840


In [0]:
def display_target_data(sql_disqry):
    try:
            pdf = spark.sql(sql_disqry)
            pd_df = pdf.toPandas()
            display(pd_df)
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  

In [0]:
def tar_auto_gen_qry(qry):
    try:
        adres = 'select '
        pd_df = spark.sql(qry)
        ow_list = pd_df.select('col_name', 'data_type').collect()
        sql_data = ow_list[0:7]
        if sql_data:
            for val in sql_data:
                col,dt = val
                if dt == 'bigint':
                    qry = "count("+col+") as "+col+"CNT,min("+col+")as "+col+"MIN,max("+col+")as "+col+"MAX,sum("+col+")as "+col+"SUM,FLOOR(avg("+col+"))as "+col+"AVG,"
                elif dt == 'string':
                    qry = "count(distinct(trim(lower("+col+")))) as "+col+"CNT,min(trim(lower("+col+")))as "+col+"MIN,max(trim(lower("+col+")))as "+col+"MAX,sum(length(trim(lower("+col+"))))as "+col+"SUM,FLOOR(avg(length(trim(lower("+col+")))))as "+col+"AVG,"
                else:
                    qry = "count(UNIX_TIMESTAMP("+col+")) as "+col+"CNT,min(UNIX_TIMESTAMP("+col+"))as "+col+"MIN,max(UNIX_TIMESTAMP("+col+"))as "+col+"MAX,sum(UNIX_TIMESTAMP("+col+"))as "+col+"SUM,FLOOR(avg(UNIX_TIMESTAMP("+col+")))as "+col+"AVG,"
                adres += qry
            rmres = adres.rstrip(',')
            sql_disqry = ''.join((rmres," from "+Source_Table_Name+""))
            return sql_disqry
        else:
            print('No Record')
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  

In [0]:
qry =("DESCRIBE TABLE {}").format(Source_Table_Name)
sql_disqry = tar_auto_gen_qry(qry)

In [0]:
display_target_data(sql_disqry)

Emp_IdCNT,Emp_IdMIN,Emp_IdMAX,Emp_IdSUM,Emp_IdAVG,Emp_nameCNT,Emp_nameMIN,Emp_nameMAX,Emp_nameSUM,Emp_nameAVG,DesignationCNT,DesignationMIN,DesignationMAX,DesignationSUM,DesignationAVG,DepartmentCNT,DepartmentMIN,DepartmentMAX,DepartmentSUM,DepartmentAVG,CityCNT,CityMIN,CityMAX,CitySUM,CityAVG,SalaryCNT,SalaryMIN,SalaryMAX,SalarySUM,SalaryAVG,Joining_DateCNT,Joining_DateMIN,Joining_DateMAX,Joining_DateSUM,Joining_DateAVG
15,4,21,194,12,15,amit desai,suresh ahir,175,11,6,admin,manager,85,5,2,hr,it,30,2,4,bardoli,surat,87,5,15,18690,56000,497284,33152,15,978307200,1427846400,18399657600,1226643840
